In [ ]:
'''
    File name: hklandsd_geocode_api.py
    Author: Steve Lau
    Date created: 10/05/2022
    Date last modified: 10/05/2022
    Python Version: 3.7
'''

In [2]:
# Setting up libraries
import pandas as pd
import requests
import urllib
import json
import fiona
from shapely.geometry import MultiPoint, Point, Polygon, shape
from shapely.geometry.polygon import Polygon

In [3]:
# Defining input and output files
input_filename = "...\input_filename.csv"
# Defining output file here.
output_filename = "...\\output_filename.csv"

In [ ]:
# Reading data to a Pandas Dataframe
df = pd.read_csv(input_filename, encoding="utf-8").copy()

In [5]:
# Concatenating "Building Name" and "Address" fields into a list.
# addresses = (df["Building Name"]+ ", " + df["Address"]).tolist()
addresses = df["Site_Address"].tolist()
districts = df["district_correct"].tolist()

In [6]:
# All looped values will be in dict format - converting to list by creating two empty lists for each attribute
name = []
address = []
xcoords = []
ycoords = []

# Looping each row of addresses to get the x and y coordinates.
for a, d in zip(addresses, districts):
    # Set up your Geocoding url.
    base_url = "https://geodata.gov.hk/gs/api/v1.0.0/locationSearch"
    endpoint = f"{base_url}?q={urllib.parse.quote_plus(str(a))}"
    # Ping for the results.
    res = requests.get(endpoint)
    # Results will be in JSON format - convert to dict using json functionality.
    data = json.loads(res.content)
    
    if len(data) == 0:
        output = {
            name.append("None"),
            address.append("None"),
            xcoords.append("None"),
            ycoords.append("None")
        }
        
    else:        
        shp_dir = fiona.open(f"F:\\python\\geospatial_scripts\\vector_translate\\ESRI Shapefile\HKGS_Dataset_2016-Population-By-census-Statistics-By-District-Council-District_2019-12-05-0800-00_fullset\\{d}.shp")
        multi = next(iter(shp_dir))

        for i in data:
            r_name = i["nameZH"]
            r_address = i["addressZH"]
            r_x = i["x"]
            r_y = i["y"]
            point = Point(r_x, r_y)
            check_i = point.within(shape(multi['geometry']))
            if check_i is not True:
                if i == data[-1]:
                    output = {
                        name.append("None"),
                        address.append("None"),
                        xcoords.append("None"),
                        ycoords.append("None")
                    }
                else:
                    continue
            else:
                name.append(r_name)
                address.append(r_address)
                xcoords.append(r_x)
                ycoords.append(r_y)
                break

In [ ]:
# Creating new columns for each attribute and grab those values
df["gc_name"] = name
df["gc_address"] = address
df["coord_x"] = xcoords
df["coord_y"] = ycoords

# Previewing dataframe in Jupyter Notebook/Jupyter Lab
df.head(10)

In [8]:
# Outputing dataframe into CSV output file
pd.DataFrame(df).to_csv(output_filename, encoding="utf_8_sig")